#### SQL info

In [ ]:
window function and group by are not used in the same query, can be used with cte. like define a cte and perfrom group by operation and the use that cte table to perfom window function operation.

#### [Q3-Premium Customers](https://100daysofsql.com/new-courses/1/coding-test/4)

Namastekart, an e-commerce company want to start special reward program for their premium customers.  The customers who have placed more number of orders than the average number of orders placed by each customers are considered as premium customers.

Write an SQL to find the list of premium customers along with the number of orders placed by each of them. 

Table: orders (primary key : order_id)

column name	datatype
order_id	int
order_date	date
customer_name	varchar(20)
sales	int
 

Example Input

orders

order_id	order_date	customer_name	sales
1	2023-01-01	Alexa	1239
2	2023-01-02	Alexa	1239
3	2023-01-03	Alexa	1239
4	2023-01-03	Alexa	1239
5	2023-01-01	Ramesh	1239
6	2023-01-02	Ramesh	1239
7	2023-01-03	Ramesh	1239
8	2023-01-03	Neha	1200
9	2023-01-03	Subhash	100
10	2023-01-03	Subhash	230
 

Expected Output:

customer_name	no_of_orders
Alexa	4
Ramesh	3
Explanation: There are total 10 orders placed by 4 customers so the average number order placed by each customers is 10/4 = 2.5 . Alexa and ramesh have done more than 2.5 orders so they are in the output. 

#my_sol

with cte as 
(select count(*)/count(distinct(customer_name)) as avg_orders from orders)

-- select * from cte
select customer_name, count(*) as no_of_orders
from orders, cte
group by customer_name
having count(*)>cte.avg_orders

#provided_sol

select customer_name,count(*) as no_of_orders 
from orders
group by customer_name having count(*)>(select count(*)*1.0/count(distinct customer_name) as avg_no_of_orders  from orders)

**chat gpt**

In SQL, when you use a comma between two tables in the `FROM` clause, it is creating a Cartesian product (cross join) between those tables. This means that every row from the first table is paired with every row from the second table. In your query:

```sql
select customer_name, count(*) as no_of_orders
from orders, cte
group by customer_name
having count(*) > cte.avg_orders
```

The `orders` table and the common table expression (CTE) `cte` are combined using a cross join. Since `cte` is expected to return only one row (it contains an aggregate calculation), the result is effectively adding the `avg_orders` value to every row from the `orders` table. 

Here's a step-by-step breakdown of the query:

1. The CTE `cte` is defined, which calculates the average number of orders per customer:
   ```sql
   with cte as 
   (select count(*)/count(distinct(customer_name)) as avg_orders from orders)
   ```
   
2. The main query:
   ```sql
   select customer_name, count(*) as no_of_orders
   from orders, cte
   group by customer_name
   having count(*) > cte.avg_orders
   ```
   - `from orders, cte`: This creates a Cartesian product of `orders` and the single row from `cte`. Because `cte` returns a single row, each row of `orders` is effectively augmented with the `avg_orders` value.
   - `group by customer_name`: This groups the result by `customer_name`.
   - `having count(*) > cte.avg_orders`: This filters the groups to only include customers whose number of orders is greater than the average number of orders per customer.

So, while a join is not explicitly specified, the cross join ensures that each row of `orders` has access to the `avg_orders` value from the CTE. This allows the `having` clause to compare the count of orders for each customer against this average.

If the CTE were to return more than one row, this query would not work correctly because the Cartesian product would multiply the rows from `orders` by the number of rows in the CTE, leading to incorrect results. In this case, since the CTE is designed to return exactly one row, the query functions as intended.

#### [4 - LinkedIn Top Voice](https://100daysofsql.com/new-courses/1/coding-test/5)

LinkedIn is a professional social networking app. They want to give top voice badge to their best creators to encourage them to create more quality content. A creator qualifies for the badge if he/she satisfies following criteria. 

1- Creator should have more then 50k followers.

2- Creator should have more than 100k impressions on the posts that they published in the month of Dec-2023.

3- Creator should have published atleast 3 posts in Dec-2023.

Write a SQL to get the list of top voice creators name along with no of posts and impressions by them in the month of Dec-2023.

Table: creators(primary key : creator_id)

column name	data type 
creator_id 	int
creator_name 	varchar(20)
followers 	int

Table: posts(primary key : post_id)

column name	data type 
creator_id 	int
post_id 	varchar(3)
publish_date 	date
impressions 	int
 

Example output:

creator_name	no_of_posts	 total_impressions
Ankit Bansal	3	132000
 

In [ ]:
my_sol

select c.creator_name, count(*) as no_of_posts, sum(p.impressions) as total_impressions
from creators c join posts p on c.creator_id=p.creator_id
where c.followers>50000 and 
strftime('%Y%m', p.publish_date) = '202312'
group by c.creator_name
having sum(impressions)>100000 and 
count(*)>=3

site_sol
select c.creator_name,count(p.post_id) as no_of_posts, sum(p.impressions) as total_impressions
from creators c
inner join posts p on c.creator_id=p.creator_id
where c.followers>50000 and strftime('%Y%m', p.publish_date)='202312'
group by c.creator_name,c.followers
having sum(p.impressions)>100000 and count(p.post_id)>=3 

#### [5 - CIBIL Score](https://100daysofsql.com/new-courses/1/coding-test/6)

CIBIL score, often referred to as a credit score, is a numerical representation of an individual's creditworthiness.While the exact formula used by credit bureaus like CIBIL may not be publicly disclosed and can vary slightly between bureaus, the following are some common factors that typically influence the calculation of a credit score:

Payment History: This accounts for the largest portion of your credit score. It includes factors such as whether you pay your bills on time, any late payments, defaults, bankruptcies, etc. Assume this accounts for 70 percent of your credit score.

Credit Utilization Ratio: This is the ratio of your credit card balances to your credit limits. Keeping this ratio low (ideally below 30%) indicates responsible credit usage. Assume it accounts for 30% of your score and below logic to calculate it: 

           Utilization below 30% = 1

           Utilization between 30% and 50% = 0.7

           Utilization above 50% = 0.5

Assume that we have credit card bills data for March 2023 based on that we need to calculate credit utilization ratio.

 

Final Credit score formula = (on_time_loan_or_bill_payment)/total_bills_and_loans * 70 + Credit Utilization Ratio * 30 

 

Table: customers(primary key : customer_id )

column name	data type 
customer_id 	int
credit_limit 	int
 

Table: loans(primary key : loan_id )

column name	data type 
customer_id 	int
loan_id 	int
loan_due_date 	date
 

Table: credit_card_bills(primary key : bill_id )

column name	data type 
customer_id 	int
bill_id 	int
bill_due_date 	date
balance_amount 	int
 

Table: customer_transactions(primary key : loan_bill_id )

column name	data type 
loan_bill_id 	int
transaction_type 	varchar(10)
transaction_date	date
 

Expected Output: 

customer_id	cibil_score
1	82.5
2	91.0

#### [11 - New and Repeat Customers](https://100daysofsql.com/new-courses/1/coding-test/12)

In [ ]:
NamasteKart an ecommerce company wants to build a very imporant business metrics where they want to track on daily basis how many new and repeat customers are purchasing products from their website. A new customer is defined when he purchased anything for the first time from the website and repeat customer is someone who has done atleast one purchase in the past.

Table: customer_orders (primary key : order_id)

column name	datatype
order_id	int
order_date	date
customer_id 	int
order_amount 	int

Expected Output:

order_date	new_customers	repeat_customers
2022-01-01	3	0
2022-01-02	2	1
2022-01-03	1	2
 

In [ ]:
-- Create the customer_orders table
CREATE TABLE customer_orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT,
    order_amount INT
);

-- Insert data into the customer_orders table
INSERT INTO customer_orders (order_id, customer_id, order_date, order_amount) VALUES
(1, 100, '2022-01-01', 2000),
(2, 200, '2022-01-01', 2500),
(3, 300, '2022-01-01', 2100),
(4, 100, '2022-01-02', 2000),
(5, 400, '2022-01-02', 2200),
(6, 500, '2022-01-02', 2700),
(7, 100, '2022-01-03', 3000),
(8, 400, '2022-01-03', 1000),
(9, 600, '2022-01-03', 3000);


In [ ]:
with cte as
(select * ,
min(order_date) over(partition by customer_id ) as first_order
from customer_orders
)

select 
order_date,
sum(case when order_date=first_order then 1 else 0 end) as new_customers,
sum(case when order_date!=first_order then 1 else 0 end) as repeat_customers
from cte
group by order_date

In [ ]:
# provided solution

with first_order_date as 
(select customer_id,min(order_date) as first_order 
from customer_orders 
group by customer_id
)
select co.order_date 
,sum(case when co.order_date=fod.first_order then 1 else 0 end) as new_customers
,sum(case when co.order_date>fod.first_order then 1 else 0 end) as repeat_customers
from customer_orders co 
inner join first_order_date fod on co.customer_id=fod.customer_id
group by co.order_date 

#### [13 - Trending Products](https://100daysofsql.com/new-courses/1/coding-test/14)

In [ ]:
-- code to insert the orders table

-- Step 1: Create the table
CREATE TABLE orders (
    order_month DATE,
    product_id VARCHAR(5),
    sales INT,
    PRIMARY KEY (order_month, product_id)
);


-- Step 2: Insert the data
INSERT INTO orders (order_month, product_id, sales) VALUES
('2023-01-01', 'p1', 100),
('2023-01-01', 'p2', 500),
('2023-02-01', 'p1', 700),
('2023-02-01', 'p2', 300),
('2023-03-01', 'p1', 900),
('2023-03-01', 'p2', 700),
('2023-04-01', 'p1', 2000),
('2023-04-01', 'p2', 1100);


In [ ]:
with cte as 
(select *,
lag(sales,1) over(partition by product_id order by order_month)  as prev1,
lag(sales,2) over(partition by product_id order by order_month) as prev2,
lag(sales,1) over(partition by product_id order by order_month)+lag(sales,2) over(partition by product_id order by order_month) as prev_total
from orders)

select order_month,product_id
from cte
where prev_total is not null and prev_total<sales;

In [ ]:
with cte2 as
(select *,
sum(sales) over(partition by product_id order by order_month rows between 2 preceding and 1 preceding) as prev_total

from orders)

select 
order_month, product_id

from cte2
where sales>prev_total and order_month not in (select distinct order_month from orders order by order_month asc limit 2)


In [ ]:
-- provided solution
with cte as (
select * 
,sum(sales) over(partition by product_id order by order_month rows between 2 preceding and 1 preceding) as last2
,row_number() over(partition by product_id order by order_month) as rn
from orders 
)
select order_month,product_id from cte 
where rn>=3 and sales > last2

#### [14 - Uber Rides](https://100daysofsql.com/new-courses/1/coding-test/15)

In [ ]:
code to create the table 

-- Create the drivers table
CREATE TABLE drivers (
    id VARCHAR(10),
    start_time TIME,
    end_time TIME,
    start_loc VARCHAR(1),
    end_loc VARCHAR(1),
    PRIMARY KEY (id, start_time)
);

-- Insert data into the drivers table
INSERT INTO drivers (id, start_time, end_time, start_loc, end_loc) VALUES
('dri_1', '09:00:00', '09:30:00', 'a', 'b'),
('dri_1', '09:30:00', '10:30:00', 'b', 'c'),
('dri_1', '11:00:00', '11:30:00', 'd', 'e'),
('dri_1', '12:00:00', '12:30:00', 'f', 'g'),
('dri_1', '13:30:00', '14:30:00', 'c', 'h'),
('dri_2', '12:15:00', '12:30:00', 'f', 'g'),
('dri_2', '13:30:00', '14:30:00', 'c', 'h');


In [ ]:
with cte as
(select *,
lead(start_time,1) over(partition by id order by start_time) as nxt_ride,
lead(start_loc,1) over(partition by id order by start_time) as nxt_start

from drivers)

select id, count(*) as total_rides,
sum(case when end_time=nxt_ride and end_loc=nxt_start then 1 else 0 end) as profit_rides
from cte
group by id


In [ ]:
-- provided solution

with cte as (
select * 
, lag(end_time,1) over(partition by id order by start_time) as prev_end_time
, lag(end_loc,1) over(partition by id order by start_time) as prev_end_loc
from drivers
)
select id, count(*) as total_rides , 
sum(case when start_time = prev_end_time and start_loc=prev_end_loc then 1 else 0 end) as profit_rides
from cte 
group by id

#### [15 - The United States of America](https://100daysofsql.com/new-courses/1/coding-test/16)

In [ ]:
''' In some poorly designed UI applications, there's often a lack of data input restrictions. For instance, in a free text field for the country, users might input variations such as 'USA,' 'United States of America,' or 'US.'

Suppose we have survey data from individuals in the USA about their job satisfaction, rated on a scale of 1 to 5. Write a SQL query to count the number of respondents for each rating on the scale. Additionally, include the country name in the format that occurs most frequently in that scale.

Table: survey (primary key : name)

column name	datatype
name 	varchar(10)
job_satisfaction 	int
country 	varchar(20)

Expected output:

job_satisfaction | country | number_of_respondents
4	USA	3
5	United States	4

'''

In [ ]:
-- Create the survey table
CREATE TABLE survey (
    name VARCHAR(10) PRIMARY KEY,
    job_satisfaction INT,
    country VARCHAR(20)
);

-- Insert data into the survey table
INSERT INTO survey (name, job_satisfaction, country) VALUES
('Alex', 4, 'USA'),
('Saurabh', 5, 'US'),
('Mark', 4, 'United States'),
('Shane', 4, 'USA'),
('Kim', 5, 'United States'),
('Joe', 5, 'USA'),
('Mira', 5, 'United States');


In [ ]:
with cte as (
select country,job_satisfaction,count(*) as cnt 
from survey
group by country,job_satisfaction
)
, cte2 as (
select *, sum(cnt) over(partition by job_satisfaction) as number_of_respondents
,max(cnt) over(partition by job_satisfaction) as max_cnt
from cte
)
select job_satisfaction,country,number_of_respondents from cte2 
where cnt=max_cnt


#### [16 - Product Recommendation](https://100daysofsql.com/new-courses/1/coding-test/17)

In [ ]:
'''
Product recommendation. Just the basic type (“customers who bought this also bought…”). That, in its simplest form, is an outcome of basket analysis. Write a SQL to find the product pairs along with the count of times they have been purchased together. Based on the history ecommerce website can recommend products to new user.

Table: orders (primary key : order_id)

column name	datatype
order_id	int
customer_id 	int
product_id 	varchar(2)

Expected Output:

product_1	product_2	purchase_frequency
p2	p1	2
p3	p1	1
p3	p2	1
p4	p1	1
p4	p2	1
Note : In the output product_1 values should be greater than product_2 values. eg: pair p1,p2 or p2,p1 represent same thing but we need to show p2,p1 in the output.

'''

In [ ]:
CREATE TABLE orders (
    order_id INT NOT NULL,
    customer_id INT NOT NULL,
    product_id VARCHAR(2) NOT NULL,
    PRIMARY KEY (order_id, product_id)
);

INSERT INTO orders (order_id, customer_id, product_id) VALUES
(1, 1, 'p1'),
(1, 1, 'p2'),
(1, 1, 'p3'),
(2, 2, 'p1'),
(2, 2, 'p2'),
(2, 2, 'p4'),
(3, 1, 'p5');

In [ ]:
select 
    o1.product_id as product_1, o2.product_id as product_2,
    count(*) as purchase_frequency
from 
    orders o1 
    join orders o2 on o1.order_id=o2.order_id
where 
    o1.product_id>o2.product_id
group by 
    o1.product_id, o2.product_id

#### [17 - Account Balance](https://100daysofsql.com/new-courses/1/coding-test/18)

In [ ]:
-- You are given a list of users and their opening account balance along with the trasaction done by them. Write a SQL to calculate their account balance at the end of all the transactions. Please note that users can do transactions among themselves as well. 

-- Table: users(primary key : user_id)

-- column name	datatype
-- user_id 	int
-- username 	varchar(10)
-- opening_balance 	int
 

-- Table: transactions (primary key : id)

-- column name	datatype
-- id	int
-- from_userid 	int
-- to_userid  	int
-- amount 	int

-- Expected Output:

-- user_id	final_balance
-- Ankit	2000
-- Rahul	10200
-- Amit	2800
-- Agam	7500

In [ ]:
-- Creating the users table
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    username VARCHAR(10) NOT NULL,
    opening_balance INTEGER NOT NULL
);

-- Creating the transactions table
CREATE TABLE transactions (
    id INTEGER PRIMARY KEY,
    from_userid INTEGER NOT NULL,
    to_userid INTEGER NOT NULL,
    amount INTEGER NOT NULL,
    FOREIGN KEY (from_userid) REFERENCES users(user_id),
    FOREIGN KEY (to_userid) REFERENCES users(user_id)
);

-- Inserting data into users table
INSERT INTO users (user_id, username, opening_balance) VALUES
(100, 'Ankit', 1000),
(101, 'Rahul', 9000),
(102, 'Amit', 5000),
(103, 'Agam', 7500);

-- Inserting data into transactions table
INSERT INTO transactions (id, from_userid, to_userid, amount) VALUES
(1, 100, 102, 500),
(2, 102, 101, 700),
(3, 101, 102, 600),
(4, 102, 100, 1500),
(5, 102, 101, 800),
(6, 102, 101, 300);


#### [18 - boAt Lifestyle](https://100daysofsql.com/new-courses/1/coding-test/19)

In [ ]:
-- boAt Lifestyle is focusing on influencer marketing to build and scale their brand. They want to partner with power creators for their upcoming campaigns. The creators should satisy below conditions to qualify :

-- 1- They should have 100k+ followers on atleast 2 social media platforms and

-- 2- They should have atleast 50k+ views on their latest YouTube video.

-- Write an SQL to get creator id and name satisfying above conditions.

-- Table: creators(primary key : order_id)

-- column name	datatype
-- id 	int
-- name 	varchar(10)
-- platform 	varchar(10)
-- followers 	int

-- Table: youtube_videos(primary key : order_id)

-- column name	datatype
-- id 	int
-- creator_id 	int
-- publish_date 	date
-- views	int
 

-- Expected Output:

-- id	name
-- 102	Dhruv

In [ ]:
-- Creating the creators table
CREATE TABLE creators (
    id INTEGER NOT NULL,
    name VARCHAR(10) NOT NULL,
    platform VARCHAR(10) NOT NULL,
    followers INTEGER NOT NULL,
    PRIMARY KEY (id, platform)
);

-- Creating the youtube_videos table
CREATE TABLE youtube_videos (
    id INTEGER PRIMARY KEY,
    creator_id INTEGER NOT NULL,
    publish_date DATE NOT NULL,
    views INTEGER NOT NULL,
    FOREIGN KEY (creator_id) REFERENCES creators(id)
);

-- Inserting data into creators table
INSERT INTO creators (id, name, platform, followers) VALUES
(100, 'Ankit', 'YouTube', 90000),
(100, 'Ankit', 'LinkedIn', 150000),
(101, 'Warikoo', 'YouTube', 500000),
(101, 'Warikoo', 'LinkedIn', 600000),
(101, 'Warikoo', 'Instagram', 800000),
(102, 'Dhruv', 'LinkedIn', 60000),
(102, 'Dhruv', 'YouTube', 900000),
(102, 'Dhruv', 'Instagram', 800000);

-- Inserting data into youtube_videos table
INSERT INTO youtube_videos (id, creator_id, publish_date, views) VALUES
(1, 100, '2024-01-01', 52000),
(2, 100, '2024-01-06', 62000),
(3, 101, '2024-01-05', 59000),
(4, 101, '2024-01-07', 22000),
(5, 102, '2024-01-05', 70000),
(6, 102, '2024-01-09', 90000);


#### [19 - Dynamic Pricing](https://100daysofsql.com/new-courses/1/coding-test/20)

In [ ]:
-- You are given a products table where a new row is inserted every time the price of a product changes. Additionally, there is a transaction table containing details such as order_date and product_ID for each order. Write an SQL query to calculate the total sales value for each product, considering the cost of the product at the time of the order date.

-- Table: products(primary key : product_id)

-- column name	datatype
-- product_id	int
-- price_date	date
-- price	int

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- product_id	int

-- Expected output:

-- product_id	total_sales
-- 100	510
-- 101	3450

In [ ]:
-- Creating the products table
CREATE TABLE products (
    product_id INTEGER NOT NULL,
    price_date DATE NOT NULL,
    price INTEGER NOT NULL,
    PRIMARY KEY (product_id, price_date)
);

-- Creating the orders table
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    order_date DATE NOT NULL,
    product_id INTEGER NOT NULL,
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
-- Inserting data into products table
INSERT INTO products (product_id, price_date, price) VALUES
(100, '2024-01-01', 150),
(100, '2024-01-21', 170),
(100, '2024-02-01', 190),
(101, '2024-01-01', 1000),
(101, '2024-01-27', 1200),
(101, '2024-02-05', 1250);

-- Inserting data into orders table
INSERT INTO orders (order_id, order_date, product_id) VALUES
(1, '2024-01-05', 100),
(2, '2024-01-21', 100),
(3, '2024-02-20', 100),
(4, '2024-01-07', 101),
(5, '2024-02-04', 101),
(6, '2024-02-05', 101);


In [ ]:
with cte as
(
    select 
        *, julianday(order_date)-julianday(price_date) as days_diff
    from 
        orders o 
        left join products p on o.order_date>=p.price_date and o.product_id=p.product_id
)
, cte2 as 
(
    select 
        *, dense_rank() over(partition by product_id, order_date order by days_diff asc) as dr
    from 
        cte
)
select 
    product_id, sum(price) as total_sales
from 
    cte2
where 
    dr=1
group by 
    product_id
-- my soln

In [ ]:
with price_range as (
select *
, date(lead(price_date,1,'9999-12-31') over(partition by product_id order by price_date),'-1 days') as price_end_date
 from products
)
select p.product_id , sum(p.price) as total_sales
from orders o 
inner join price_range p on o.product_id=p.product_id 
and o.order_date between p.price_date and p.price_end_date
group by p.product_id;
-- provided soln

#### [20 - Income Tax Returns](https://100daysofsql.com/new-courses/1/coding-test/21)

In [ ]:
-- Income tax returns are supposed to file every year before due date announce by govt every year . Goverment wants to find out people who have filed late returns or people who have completely skipped filing returns. 

-- Write an SQL to list all the users along with the year for which they have done late filing or missed filing the returns. Create a comment column as well specifying if its a 'late return' or  'missed'.

-- Table: income_tax_dates(primary key : financial_year)

-- column name	datatype
-- financial_year 	varchar(4)
-- file_start_date 	date
-- file_due_date 	date
 

-- Table: users (primary key : user_id,financial_year)

-- column name	datatype
-- user_id 	int
-- financial_year 	varchar(4)
-- return_file_date 	date
 


-- Expected Output:

-- user_id	financial_year	comment
-- 1	FY21	late return
-- 1	FY22	missed
-- 2	FY23	late return
 

In [ ]:
-- Creating the income_tax_dates table
CREATE TABLE income_tax_dates (
    financial_year VARCHAR(4) PRIMARY KEY,
    file_start_date DATE NOT NULL,
    file_due_date DATE NOT NULL
);

-- Creating the users table
CREATE TABLE users (
    user_id INTEGER NOT NULL,
    financial_year VARCHAR(4) NOT NULL,
    return_file_date DATE NOT NULL,
    PRIMARY KEY (user_id, financial_year),
    FOREIGN KEY (financial_year) REFERENCES income_tax_dates(financial_year)
);
-- Inserting data into income_tax_dates table
INSERT INTO income_tax_dates (financial_year, file_start_date, file_due_date) VALUES
('FY20', '2020-05-01', '2020-08-31'),
('FY21', '2021-06-01', '2021-09-30'),  -- Changed '2021-09-31' to '2021-09-30' to correct the date
('FY22', '2022-05-05', '2022-08-29'),
('FY23', '2023-05-05', '2023-08-31');

-- Inserting data into users table
INSERT INTO users (user_id, financial_year, return_file_date) VALUES
(1, 'FY20', '2020-05-10'),
(1, 'FY21', '2021-10-10'),
(1, 'FY23', '2023-08-20'),
(2, 'FY20', '2020-05-15'),
(2, 'FY21', '2021-09-10'),
(2, 'FY22', '2022-08-20'),
(2, 'FY23', '2023-10-10');


In [ ]:
-- my soln
with all_users_years as 
(
select 
u.user_id,it.financial_year,it.file_due_date

from users u , income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)

select * from 
(select  -- *
a.user_id, a.financial_year, 
case when u.return_file_date>a.file_due_date then 'late return'
when u.return_file_date<a.file_due_date then 'on_time'
else 'missed' end as comment
from all_users_years a left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year) a
where comment !='on_time'

In [ ]:
-- provided solution
with all_users_years as 
(
select u.user_id,it.financial_year,it.file_due_date
from users u cross join income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)
select a.user_id,a.financial_year
,case when u.return_file_date > a.file_due_date then 'late return' else 'missed' end as comment
from all_users_years a 
left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year 
where u.return_file_date > a.file_due_date or u.return_file_date is null;